## Import Data

In [14]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

        

In [15]:
INPUT_FOLDER = "preproc"
OUTPUT_FOLDER = "enrich"

In [16]:
key = "us_companies"
df_us_companies = pd.read_csv(f"data/{INPUT_FOLDER}/{key}.csv")
df_us_companies.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5020 entries, 0 to 5019
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   SimFinId                       5020 non-null   int64  
 1   Company Name                   5020 non-null   object 
 2   IndustryId                     5020 non-null   float64
 3   ISIN                           5020 non-null   object 
 4   End of financial year (month)  5020 non-null   float64
 5   Number Employees               5020 non-null   float64
 6   Business Summary               5020 non-null   object 
 7   Market                         5020 non-null   object 
 8   CIK                            5020 non-null   float64
 9   Main Currency                  5020 non-null   object 
dtypes: float64(4), int64(1), object(5)
memory usage: 392.3+ KB


In [17]:
key = "us-shareprices-daily"
df_us_share_price = pd.read_csv(f"data/{INPUT_FOLDER}/{key}.csv", parse_dates=["Date"])
df_us_share_price.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5815913 entries, 0 to 5815912
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   Ticker              5815913 non-null  object        
 1   SimFinId            5815913 non-null  int64         
 2   Date                5815913 non-null  datetime64[ns]
 3   Open                5815913 non-null  float64       
 4   High                5815913 non-null  float64       
 5   Low                 5815913 non-null  float64       
 6   Close               5815913 non-null  float64       
 7   Adj. Close          5815913 non-null  float64       
 8   Volume              5815913 non-null  int64         
 9   Dividend            5815913 non-null  float64       
 10  Shares Outstanding  5815913 non-null  float64       
dtypes: datetime64[ns](1), float64(7), int64(2), object(1)
memory usage: 488.1+ MB


In [18]:
#join df_us_share_price with df_us_companies on SimFinId
df = pd.merge(df_us_share_price, df_us_companies, on="SimFinId", how="left")

df.head()

,Ticker,SimFinId,Date,Open,High,Low,Close,Adj. Close,Volume,Dividend,Shares Outstanding,Company Name,IndustryId,ISIN,End of financial year (month),Number Employees,Business Summary,Market,CIK,Main Currency
0,A,45846,2019-04-12,81.43,82.06,80.90,80.98,77.77,1249295,0.0,317515869.0,AGILENT TECHNOLOGIES INC,106001.0,US00846U1016,10.0,16400.0,Agilent Technologies Inc is engaged in life sc...,us,1090872.0,USD
1,A,45846,2019-04-15,81.00,81.13,79.91,80.40,77.22,1627268,0.0,317515869.0,AGILENT TECHNOLOGIES INC,106001.0,US00846U1016,10.0,16400.0,Agilent Technologies Inc is engaged in life sc...,us,1090872.0,USD
2,A,45846,2019-04-16,80.82,80.96,77.19,77.55,74.48,3441597,0.0,317515869.0,AGILENT TECHNOLOGIES INC,106001.0,US00846U1016,10.0,16400.0,Agilent Technologies Inc is engaged in life sc...,us,1090872.0,USD
3,A,45846,2019-04-17,78.15,78.32,74.46,75.43,72.44,4471971,0.0,317515869.0,AGILENT TECHNOLOGIES INC,106001.0,US00846U1016,10.0,16400.0,Agilent Technologies Inc is engaged in life sc...,us,1090872.0,USD
4,A,45846,2019-04-18,75.73,76.54,75.31,76.17,73.16,2874195,0.0,317515869.0,AGILENT TECHNOLOGIES INC,106001.0,US00846U1016,10.0,16400.0,Agilent Technologies Inc is engaged in life sc...,us,1090872.0,USD


## Import Function

In [19]:
#function
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error

def train_xgboost_model(data, simfin_id, last_date):
    # Filter the DataFrame to get the last 60 days of data
    data = data[['SimFinId', 'Date', 'Close']]
    data = data.set_index(['SimFinId', 'Date'])
    last_date = pd.to_datetime(last_date)
    sixty_days_prior = last_date - pd.DateOffset(days=60)
    data = data[(data.index.get_level_values('Date') > sixty_days_prior) & (data.index.get_level_values('Date') <= last_date)]

    # Rename close to target
    data = data.rename(columns={'Close': 'target'})
    # Filter the DataFrame based on the provided SimFinId
    data_filtered = data[data.index.get_level_values('SimFinId') == simfin_id]
    
    # Extract useful features from the Date column
    data_filtered['year'] = data_filtered.index.get_level_values('Date').year
    data_filtered['month'] = data_filtered.index.get_level_values('Date').month
    data_filtered['day'] = data_filtered.index.get_level_values('Date').day
    data_filtered['dayofweek'] = data_filtered.index.get_level_values('Date').dayofweek
    data_filtered['lag1'] = data_filtered['target'].shift(1)

    # Prepare the data
    X = data_filtered.drop(columns=['target'])
    y = data_filtered['target']
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train an XGBoost model
    model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5)
    model.fit(X_train, y_train)
    
    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate the model
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    
    # Predict the next 15 days
    future_dates = pd.date_range(start=last_date + pd.DateOffset(days=1), periods=15, freq='B')  # Business days
    future_data = pd.DataFrame(index=pd.MultiIndex.from_product([[simfin_id], future_dates], names=['SimFinId', 'Date']))
    future_data['year'] = future_data.index.get_level_values('Date').year
    future_data['month'] = future_data.index.get_level_values('Date').month
    future_data['day'] = future_data.index.get_level_values('Date').day
    future_data['dayofweek'] = future_data.index.get_level_values('Date').dayofweek
    future_data['lag1'] = data_filtered['target'].iloc[-1]
    
    future_predictions = model.predict(future_data)
    future_predictions_df = pd.DataFrame({'Date': future_dates, 'Predicted_Close': future_predictions})
    
    return future_predictions_df

# Example usage
# Assuming 'data' is your DataFrame and 'SimFinId' is part of the index
# simfin_id = 59265
# date_for_model = pd.to_datetime('2024-03-04')  # Convert string to datetime
# model, train_mse, test_mse, future_predictions = train_xgboost_model(data, simfin_id, date_for_model)
# print(f"Train Mean Squared Error: {train_mse}")
# print(f"Test Mean Squared Error: {test_mse}")
# print(f"Future Predictions: {future_predictions}")

In [27]:
# Function trading strategy
def trading_strategy (future_predictions_df):
    low = prediction_df['Predicted_Close'].min()
    date_low = prediction_df['Date'][prediction_df['Predicted_Close'].idxmin()].strftime('%Y-%m-%d')
    high_after_date_low = prediction_df[prediction_df['Date'] > date_low]['Predicted_Close'].max()
    date_high_after_date_low = prediction_df['Date'][prediction_df['Predicted_Close'] == high_after_date_low].values[0]
    date_high_after_date_low = pd.to_datetime(date_high_after_date_low).strftime('%Y-%m-%d')
    percentual_change = (high_after_date_low - low) / low * 100
    days_from_low_to_high = (pd.to_datetime(date_high_after_date_low) - pd.to_datetime(date_low)).days

    if low == high_after_date_low or percentual_change < 0 or days_from_low_to_high < 0 or percentual_change < 0.5: 
        result = "Don't buy this stock, it's going nowhere!"
    else:
        result = f'Buy the stock on {date_low} and sell it on {date_high_after_date_low}, earnings up to {percentual_change:.2f}%'
    return result

## Test

In [28]:
prediction_df = train_xgboost_model(df, 59265, '2024-03-04')
prediction_df

/var/folders/0z/4yrpt4m15ps8kzs2ljkgk40r0000gn/T/ipykernel_81395/2552309633.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered['year'] = data_filtered.index.get_level_values('Date').year
/var/folders/0z/4yrpt4m15ps8kzs2ljkgk40r0000gn/T/ipykernel_81395/2552309633.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered['month'] = data_filtered.index.get_level_values('Date').month
/var/folders/0z/4yrpt4m15ps8kzs2ljkgk40r0000gn/T/ipykernel_81395/2552309633.py:23: SettingWithCopyWarnin

,Date,Predicted_Close
0,2024-03-05,418.226715
1,2024-03-06,419.447174
2,2024-03-07,420.467255
3,2024-03-08,420.276642
4,2024-03-11,415.459290
5,2024-03-12,418.792236
6,2024-03-13,419.989960
7,2024-03-14,420.050842
8,2024-03-15,415.554871
9,2024-03-18,411.280548


In [29]:
trading_strategy(prediction_df)

'Buy the stock on 2024-03-18 and sell it on 2024-03-22, earnings up to 1.32%'